<a href="https://colab.research.google.com/github/GazarovaPolina/VKFriendsGraphMaker/blob/main/graphMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
import csv
import networkx as nx


URL = 'https://api.vk.com/method/friends.get'
V = '5.131'

TOKEN = 'vk1.a.jdI3fp9XyVAj6e56eFjXbM70Smg6aw1jR5xXjUdDGosNVJp69D3YUB-pusRcoC9aVSYEoplJH335bEIUbiV_esOFebsefDHq3A1CEoBmZP8xI54p1SQhSpEB8NCGfePqqOyAsyXW76XOBSszRf4WFSspsiPJEkDBlEaKNHFiBLE1Si8-70wSM9zs_cqYzmV3'
USER_ID = '169119879'


request = f"{URL}?" \
            f"v={V}&" \
            f"access_token={TOKEN}&" \
            f"user_id={USER_ID}&"

graph = {}

print(requests.get(request).json())
data = requests.get(request).json()['response']['items']
graph[USER_ID] = data

# Создание графа
unique_ids = set()

for id in data:  # Получаем друзей по vk id, которые получили раньше
    if len(unique_ids) > 200:
        break
    try:
        request = f"{URL}?" \
                  f"v={V}&" \
                  f"access_token={TOKEN}&" \
                  f"user_id={id}&"

        data = requests.get(request).json()['response']['items']
        graph[str(id)] = data
        unique_ids.update(data)
    except KeyError:
        pass
print(len(unique_ids))

# Создание графа NetworkX
G = nx.Graph()

# Добавление рёбер в граф
for node, neighbors in graph.items():
    for neighbor in neighbors:
        G.add_edge(node, neighbor)


# print(G.nodes)

dict_closeness_centrality = {}
max = 0
nodemaxes = list()
for node in G.nodes:
    current_node_closeness_centrality = nx.closeness_centrality(G, u=node)
    dict_closeness_centrality[node] = current_node_closeness_centrality
    if dict_closeness_centrality[node] > max:
        max = dict_closeness_centrality[node]
        nodemaxes = [node]
    elif dict_closeness_centrality[node] == max:
        nodemaxes = nodemaxes.append(node)
print(f"Центральность по близости \n Максимальное значение: {max}   \n Ids: {nodemaxes}")

print("Центральность по близости измеряет, насколько быстро узел может достичь всех других узлов в графе."
      " Узел с высокой близостью имеет короткие пути ко всем другим узлам. В контексте графа друзей и друзей друзей,"
      " это означает, что узел (например, пользователь) может быстро связаться со всеми своими друзьями и их друзьями. "
      "Это полезно для определения, кто из друзей является наиболее доступным для общения и взаимодействия.\n")

dict_betweenness_centrality = {}
max = 0
nodemaxes = list()
for node in G.nodes:
    current_node_betweenness_centrality = nx.betweenness_centrality(G, normalized=True)[node]
    dict_betweenness_centrality[node] = current_node_betweenness_centrality
    if dict_betweenness_centrality[node] > max:
        max = dict_betweenness_centrality[node]
        nodemaxes = [node]
    elif dict_betweenness_centrality[node] == max:
        nodemaxes = nodemaxes.append(node)
print(f"Центральность по посреднечеству для узла: \n Максимальное значение: {max}   \n Ids: {nodemaxes}")

print("В графе друзей это может означать, что некоторые друзья могут связывать другие группы друзей,"
      " обеспечивая тем самым важные социальные связи.\n")

dict_eigenvector_centrality = {}
max = 0
nodemaxes = list()
for node in G.nodes:
    current_node_eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)[node]
    dict_eigenvector_centrality[node] = current_node_eigenvector_centrality
    if dict_eigenvector_centrality[node] > max:
        max = dict_eigenvector_centrality[node]
        nodemaxes = [node]
    elif dict_eigenvector_centrality[node] == max:
        nodemaxes = nodemaxes.append(node)
print(f"Центральность по собственному вектору для узла: \n Максимальное значение: {max}   \n Ids: {nodemaxes}")

print("Узел получает высокую оценку, если его соседи также имеют высокую центральность. "
      "В графе друзей это означает, что если у пользователя много влиятельных друзей, "
      "его собственная центральность будет выше. Это подчеркивает важность не только "
      "количества связей, но и их значимости.\n")


# Узел, для которого нужно рассчитать центральность
target_node = USER_ID

closeness_centrality = nx.closeness_centrality(G, u=target_node)
print(f'Центральность по близости для узла {target_node}: {closeness_centrality:.4f}')

betweenness_centrality = nx.betweenness_centrality(G, normalized=True)[target_node]
print(f'Центральность по посредничеству для узла {target_node}: {betweenness_centrality:.4f}')

# Расчет центральности по собственному вектору
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)[target_node]
print(f'Центральность по собственному вектору для узла {target_node}: {eigenvector_centrality:.4f}')